In [ ]:
!pip install pymupdf sentence-transformers transformers torch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from google.colab import userdata
gemini_key= userdata.get('Gemini')

In [12]:
import fitz
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline

#Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    doc.close()
    return text

#Chunk the extracted text
def chunk_text(text, chunk_size=500):
    sentences = text.split('. ')
    chunks = [' '.join(sentences[i:i+chunk_size]) for i in range(0, len(sentences), chunk_size)]
    return chunks

# Rerank the chunks based on the query (top 3)
def retrieve_relevant_chunks(query, text_chunks, chunk_embeddings, model):
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarity_scores = util.pytorch_cos_sim(query_embedding, chunk_embeddings)
    top_chunk_indices = torch.argsort(similarity_scores, descending=True).squeeze()
    top_chunks = [text_chunks[i] for i in top_chunk_indices[:3]]
    return top_chunks

def generate_response_google_gemini(query, retrieved_chunks):
    context = " ".join(retrieved_chunks)
    prompt = query + " " + context

    response = call_google_gemini_api(prompt)
    return response

def call_google_gemini_api(prompt):
    import requests

    api_url = "https://api.google.com/gemini/generate"
    headers = {"Authorization": gemini_key}
    data = {"prompt": prompt}

    response = requests.post(api_url, headers=headers, json=data)


    if response.status_code == 200:
        return response.json()['text']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return "Error generating response"

In [13]:

pdf_path = '/content/drive/My Drive/Capstone/basics_of_strength_and_conditioning_manual.pdf'  # Update with your PDF path

# Extract and chunk the PDF text
pdf_text = extract_text_from_pdf(pdf_path)
text_chunks = chunk_text(pdf_text, chunk_size=5)

#  pre-trained sentence transformer model for embedding
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # A compact and efficient transformer

# Embed  text chunks
chunk_embeddings = embedding_model.encode(text_chunks, convert_to_tensor=True)

query = "What exercises are recommended for cardiovascular health?"

# Retrieve relevant chunks based on the query
top_chunks = retrieve_relevant_chunks(query, text_chunks, chunk_embeddings, embedding_model)

# Generate a response
response = generate_response_google_gemini(query, top_chunks)

# Output
print("Generated Response from Google Gemini:")
print(response)

output_text_path = '/content/drive/My Drive/Capstone/extracted_text.txt'
with open(output_text_path, 'w', encoding='utf-8') as text_file:
    text_file.write(pdf_text)

print(f"Text data has been saved to {output_text_path}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Error: 404, <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-moz-border-imag